In [5]:


import os
import shutil
from roboflow import Roboflow


if os.path.exists("dataset"):
    shutil.rmtree("dataset")
    print(" Removed empty/broken dataset folder.")


base_dir = "submission/Part_1_Glove_Detection"
os.makedirs(f"{base_dir}/output", exist_ok=True)
os.makedirs(f"{base_dir}/logs", exist_ok=True)


print(" Downloading Dataset...")



from roboflow import Roboflow
rf = Roboflow(api_key="ynxqkPjvRzJuWC72JB3N")
project = rf.workspace("dolphin-nog9y").project("gloves-and-bare-hands-detection")
version = project.version(2)
dataset = version.download("yolov8")


found = False
for folder in os.listdir('.'):
    # Look for the folder Roboflow just made
    if os.path.isdir(folder) and "detection" in folder and folder != "submission":
        os.rename(folder, "dataset")
        print(f"Renamed '{folder}' to 'dataset'.")
        found = True
        break

if found:
    print(" Step 1 Successful. Now Step 2 will work!")
else:
    print("Error: Dataset download failed ?")



⬇️ Downloading Dataset...
loading Roboflow workspace...
loading Roboflow project...
✅ Renamed 'Gloves-and-bare-hands-detection-2' to 'dataset'.
✅ Step 1 Successful. Now Step 2 will work!


In [6]:
!pip install ultralytics

from ultralytics import YOLO
import os


data_yaml = None
for root, _, files in os.walk("dataset"):
    if "data.yaml" in files:
        data_yaml = os.path.join(root, "data.yaml")
        break

if not data_yaml:
    print("Error: data.yaml not found. ")
else:

    print(f" Starting training using config: {data_yaml}")
    model = YOLO('yolov8n.pt')

    model.train(
        data=data_yaml,
        epochs=20,
        imgsz=640,
        batch=16,
        project='submission/training_logs',
        name='glove_detection_run',
        exist_ok=True,
        verbose=True
    )
    print(" Training Complete.")

🚀 Starting training using config: dataset/data.yaml
Ultralytics 8.3.244 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=glove_detection_run, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=T

In [7]:
%%writefile submission/Part_1_Glove_Detection/detection_script.py
import argparse
import json
import os
import sys
from pathlib import Path
import cv2
from tqdm import tqdm
from ultralytics import YOLO


LABEL_MAP = {
    "surgical-gloves": "gloved_hand",
    "glove": "gloved_hand",
    "gloverotation": "gloved_hand",
    "hand": "bare_hand",
    "bare": "bare_hand"
}


COLORS = {"gloved_hand": (0, 255, 0), "bare_hand": (0, 0, 255)}

def normalize_label(raw_label):
    """Normalize dataset labels to project requirements."""
    if raw_label in LABEL_MAP:
        return LABEL_MAP[raw_label]

    if "glove" in raw_label.lower():
        return "gloved_hand"
    return "bare_hand"

def run_inference(input_dir, output_dir, model_path, conf_thres):
    input_path = Path(input_dir)
    output_path = Path(output_dir)
    logs_path = Path("submission/Part_1_Glove_Detection/logs")

    # Ensure directories exist
    output_path.mkdir(parents=True, exist_ok=True)
    logs_path.mkdir(parents=True, exist_ok=True)

    print(f"Loading model: {model_path}")
    try:
        model = YOLO(model_path)
    except Exception as e:
        print(f"Critical Error loading model: {e}")
        sys.exit(1)

    # Gather images
    valid_exts = {".jpg", ".jpeg", ".png"}
    images = [p for p in input_path.iterdir() if p.suffix.lower() in valid_exts]

    if not images:
        print("No images found in input directory.")
        return

    print(f"Processing {len(images)} images...")

    # Run Inference (Stream mode for memory efficiency)
    results = model.predict(
        source=[str(p) for p in images],
        conf=conf_thres,
        stream=True,
        verbose=False
    )

    for result in tqdm(results, total=len(images)):
        filename = os.path.basename(result.path)
        img_out = result.orig_img.copy()

        # JSON Schema
        log_entry = {
            "filename": filename,
            "detections": []
        }

        for box in result.boxes:
            # Extract Integer Coordinates
            x1, y1, x2, y2 = [int(v) for v in box.xyxy[0].tolist()]
            conf = float(box.conf[0])

            # Resolve Class Name
            raw_class = model.names[int(box.cls[0])]
            label = normalize_label(raw_class)

            # Update Log
            log_entry["detections"].append({
                "label": label,
                "confidence": round(conf, 2),
                "bbox": [x1, y1, x2, y2]
            })

            # Draw Annotation
            color = COLORS.get(label, (255, 255, 255))
            cv2.rectangle(img_out, (x1, y1), (x2, y2), color, 2)

            label_text = f"{label} {conf:.2f}"
            (w, h), _ = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
            cv2.rectangle(img_out, (x1, y1 - 20), (x1 + w, y1), color, -1)
            cv2.putText(
                img_out, label_text, (x1, y1 - 5),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1
            )

        # Save Artifacts
        cv2.imwrite(str(output_path / filename), img_out)

        json_path = logs_path / f"{Path(filename).stem}.json"
        with open(json_path, 'w') as f:
            json.dump(log_entry, f, indent=4)

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--input", required=True, help="Input folder")
    parser.add_argument("--output", default="submission/Part_1_Glove_Detection/output")
    parser.add_argument("--weights", required=True, help="Path to .pt model")
    parser.add_argument("--confidence", type=float, default=0.45)

    args = parser.parse_args()
    run_inference(args.input, args.output, args.weights, args.confidence)

Writing submission/Part_1_Glove_Detection/detection_script.py


In [8]:
%%writefile submission/Part_1_Glove_Detection/README.md
# Part 1: Glove Compliance Detection Pipeline

## Project Overview
This repository contains a computer vision solution to detect `gloved_hand` and `bare_hand` classes for safety compliance monitoring.

## Technical Implementation
*   **Model:** YOLOv8 Nano (`yolov8n`). Selected for optimal real-time performance on edge hardware.
*   **Data Source:** Roboflow Universe (Gloves & Bare Hands).
*   **Preprocessing:**
    *   Applied Transfer Learning on ~800 images.
    *   Utilized Mosaic augmentation to improve robustness.
    *   Implemented a **Label Normalization Layer** in the inference script to handle inconsistencies in the source dataset (e.g., mapping `gloverotation` to `gloved_hand`).

## Usage
**Run the detection script:**
```bash
python detection_script.py \
  --input data/images \
  --output results/ \
  --weights best.pt \
  --confidence 0.45

Writing submission/Part_1_Glove_Detection/README.md


In [10]:
import os
import shutil
import subprocess

# 1. Cleanup old outputs
if os.path.exists("submission/Part_1_Glove_Detection/output"):
    shutil.rmtree("submission/Part_1_Glove_Detection/output")
print(" Cleared old output.")

# 2. Find Images
image_source = None
for root, dirs, files in os.walk("dataset"):
    if "valid" in dirs and "images" in os.listdir(os.path.join(root, "valid")):
        image_source = os.path.join(root, "valid", "images")
        break
    # Fallback search
    if "images" in dirs:
        temp_path = os.path.join(root, "images")
        if any(f.endswith(".jpg") for f in os.listdir(temp_path)):
            image_source = temp_path
            break

# 3. Execute
if image_source:
    print(f" Processing images from: {image_source}")
    cmd = [
        "python", "submission/Part_1_Glove_Detection/detection_script.py",
        "--input", image_source,
        "--output", "submission/Part_1_Glove_Detection/output",
        "--weights", "submission/training_logs/glove_detection_run/weights/best.pt",
        "--confidence", "0.45"
    ]
    subprocess.run(cmd, check=True)
    print(" Inference Complete. Outputs generated.")
else:
    print(" Error: Could not find dataset images.")

🧹 Cleared old output.
🚀 Processing images from: dataset/valid/images
✅ Inference Complete. Outputs generated.


In [11]:
import shutil
from google.colab import files

print(" Creating final submission zip...")
shutil.make_archive('submission_final', 'zip', 'submission')

print(" Downloading file...")
files.download('submission_final.zip')



 Creating final submission zip...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>